# ANEXO

## Librerias 

Las librerias utilizadas han sido las siguientes.

In [1]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd
import seaborn as sb

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn import tree

# Preprocesado
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import statsmodels.api as sm

# Optimización de parámetros
# ------------------------------------------------------------------------------
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import f1_score, make_scorer

# Modelado
# ------------------------------------------------------------------------------
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import StackingClassifier

from sklearn.model_selection import cross_val_score

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

## Datos

Cargamos los datos que utilizaremos para realizar el proyecto. 

In [ ]:
import pandas as pd
df=pd.read_excel('dataset_escalado.xlsx')
df.head()

Eliminamos las variables cualitativas.

In [3]:
df = df.drop(['ID_PDA','SN_PDA','FECHA_FAB','FECHA_TEST','SN_BAT'], axis=1)

Esta función me permite eliminar de la base de datos los valores que son NaN y los Inf.

In [4]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [ ]:
clean_dataset(df)

Definimos las variable X junto con la variable objetivo y.

In [6]:
X = df.drop(['ESTADO_BAT'], axis=1).values
y = df['ESTADO_BAT'].values

Creamos un histograma para observar la distribución de las clases presenten en la variable objetivo. 

In [ ]:
sb.catplot('ESTADO_BAT',data=df,kind="count")

División de los datos en train y en test y aplicamos el algoritmo SMOTE a los datos de train. 

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
                                        df.drop(columns = 'ESTADO_BAT'),
                                        df['ESTADO_BAT'],
                                        random_state = 123,
                                        stratify = df['ESTADO_BAT'],
                                        train_size=0.7)
sm = SMOTE(sampling_strategy='auto', k_neighbors=2, random_state=100)
X_train, y_train = sm.fit_resample(X_train, y_train)

A continuación definimos una función para optimar los parámetros óptimos de los diferentes modelos probados.

In [10]:
def clfGrid(X, y, clasificadores, n_clas):
    res = {}
    for clf_cnt, clf in enumerate(clasificadores):
        clf.fit(X, y)     
        res[n_clas[clf_cnt]] = {}
        res[n_clas[clf_cnt]]["F1"] = clf.best_score_
        res[n_clas[clf_cnt]]["Mejor clasificador"] = clf.best_estimator_
    return res

Definimos los siguientes clasificadores junto con la métrica f1 score.

In [11]:
f1 = make_scorer(f1_score , average='weighted')

clasificadores = [GridSearchCV(estimator=LogisticRegression(), 
                               param_grid = {'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1], 
                                             'solver' : ('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'), 
                                             'max_iter' : [100, 200, 300]}, scoring=f1),
                  
                  GridSearchCV(estimator=AdaBoostClassifier(), 
                               param_grid = {'n_estimators':[50,100,200,500]}, 
                               scoring=f1),
                  
                  GridSearchCV(estimator=LinearSVC(), 
                               param_grid = {'C':[0.1,1,1.5]}, scoring=f1),
                  
                  GridSearchCV(estimator=KNeighborsClassifier(), 
                               param_grid = {'n_neighbors':[1,2,3,4,5,6], 
                                             'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'],
                                             'p':[1,2]}, scoring=f1),
                  
                  GridSearchCV(estimator=DecisionTreeClassifier(), 
                               param_grid = {'criterion':['gini','entropy'],
                                             'splitter':['random','best'],
                                             'min_samples_split':[2,4,6], 
                                             'max_depth': [2, 5, 10, 15, 20], 
                                             'min_samples_leaf': [5, 4, 3, 2, 1]}, 
                               scoring=f1),
                  
                 GridSearchCV(estimator = MLPClassifier(random_state = 1), 
                              param_grid={'activation': ('identity', 'logistic', 'tanh', 'relu'), 
                                          'solver': ('lbfgs', 'sgd', 'adam'), 'max_iter': [2, 5, 10]}, 
                              scoring=f1),
                  
                 GridSearchCV(estimator = SVC(random_state = 1), 
                              param_grid = dict(C = [1,2,4], gamma = ['scale', 'auto'], 
                                                kernel = ['poly', 'rbf', 'sigmoid']), 
                              scoring=f1)]

n_clas = ['Regressión logística', 'AdaBoost', 'SVC Lineal','Vecinos','Arboles', 'Perceptrón Multicapa', 'SVC']

In [ ]:
clfGrid(X_train,y_train, clasificadores, n_clas)

Una vez obtenidos los parámetros óptimos de los diferentes clasificadores, pasaremos a aplicar cada modelo con sus parámetros óptimos.

Definimos una función que nos proporcionará la métrica brier score.

In [13]:
def brier_multi(targets, probs):
    return np.mean(np.sum((probs - targets)**2, axis=1))

#### Modelo de Árboles de Clasificación

In [ ]:
modelo = DecisionTreeClassifier(max_depth = 10, splitter="random",random_state=10)
f1 = cross_val_score(modelo, X_train,y_train,cv=5 ,scoring = 'f1_weighted').mean()
fit = modelo.fit(X_train,y_train)
pred = modelo.predict(X_test)
probs = modelo.predict_proba(X_test)

bien = []
for i in pred:
    if i == 0:
        bien.append([1,0,0,0])
    elif i == 25:
        bien.append([0,1,0,0])
    elif i == 50:
        bien.append([0,0,1,0])
    elif i == 75:
        bien.append([0,0,0,1])
bien = np.array(bien)
print('Los valores de las métricas para dicho modelo son los siguientes: ')
print(f'El Brier Score da:',brier_multi(probs,bien))
print(f'El f1 de predict da:',f1_score(y_test,pred, average='weighted'))
print(f'F1 cross-val da(Decision Tree): {f1:.5f}')

Definimos la estructura del árbol. 

In [ ]:
fig, ax = plt.subplots(figsize=(21, 10))

print(f"Profundidad del árbol: {modelo.get_depth()}")
print(f"Número de nodos terminales: {modelo.get_n_leaves()}")

plot = plot_tree(
            decision_tree = modelo,
            feature_names = list(df.drop(['ESTADO_BAT'], axis=1)),
            filled        = True,
            impurity      = False,
            fontsize      = 7,
            ax            = ax)

Nos guardamos la estructura del árbol que nos proporciona dicho modelo. 

In [16]:
fig.savefig("árbol_clasificación.png")

Finalmente obtenemos la importacia que han tenido las variables en el modelo.  

In [ ]:
print("Importancia de los predictores en el modelo")
print("-------------------------------------------")
importancia_predictores = pd.DataFrame(
                            {'predictor': df.drop(columns = "ESTADO_BAT").columns,
                             'importancia': modelo.feature_importances_}
                            )
importancia_predictores.sort_values('importancia', ascending=False).head(11)

#### Modelo de Regresión Logística

In [ ]:
modelo =  LogisticRegression(C=1, solver='liblinear')
fit = modelo.fit(X_train,y_train)
f1 = cross_val_score(modelo, X_train,y_train,cv=5 ,scoring = 'f1_weighted').mean()
pred = modelo.predict(X_test)
probs = modelo.predict_proba(X_test)

bien = []
for i in pred:
    if i == 0:
        bien.append([1,0,0,0])
    elif i == 25:
        bien.append([0,1,0,0])
    elif i == 50:
        bien.append([0,0,1,0])
    elif i == 75:
        bien.append([0,0,0,1])
bien = np.array(bien)
print('Los valores de las métricas para dicho modelo son los siguientes: ')
print(f'El brier Score da:',brier_multi(probs,bien))
print(f'El f1 predicho da:',f1_score(y_test,pred, average='weighted'))
print(f'F1 (Regresión Logística): {f1:.5f}')

#### Modelo de K-Vecinos

In [ ]:
modelo =  KNeighborsClassifier(n_neighbors=1, p=1)
f1 = cross_val_score(modelo, X_train,y_train,cv=5 ,scoring = 'f1_weighted').mean()
fit = modelo.fit(X_train,y_train)
pred = modelo.predict(X_test)
probs = modelo.predict_proba(X_test)

bien = []
for i in pred:
    if i == 0:
        bien.append([1,0,0,0])
    elif i == 25:
        bien.append([0,1,0,0])
    elif i == 50:
        bien.append([0,0,1,0])
    elif i == 75:
        bien.append([0,0,0,1])
bien = np.array(bien)
print('Los valores de las métricas para dicho modelo son los siguientes: ')
print(f'El brier Score da:',brier_multi(probs,bien))
print(f'El f1 predicho da:',f1_score(y_test,pred, average='weighted'))
print(f'F1 (KNN): {f1:.5f}')

#### Modelo de Máquinas de Vectores Soporte

In [ ]:
modelo =  SVC(C=4, gamma='auto', random_state=1,probability=True)
f1 = cross_val_score(modelo, X_train,y_train,cv=5 ,scoring = 'f1_weighted').mean()
fit = modelo.fit(X_train,y_train)
pred = modelo.predict(X_test)
probs = modelo.predict_proba(X_test)

bien = []
for i in pred:
    if i == 0:
        bien.append([1,0,0,0])
    elif i == 25:
        bien.append([0,1,0,0])
    elif i == 50:
        bien.append([0,0,1,0])
    elif i == 75:
        bien.append([0,0,0,1])
bien = np.array(bien)
print('Los valores de las métricas para dicho modelo son los siguientes: ')
print(f'El brier Score da:',brier_multi(probs,bien))
print(f'El f1 predicho da:',f1_score(y_test,pred, average='weighted'))
print(f'F1 (SVC): {f1:.5f}')

#### Probamos Stacking con los diferentes clasificadores

In [ ]:
estimators=[('mlp',MLPClassifier(random_state=1, max_iter=1000)),('LinearSVC', LinearSVC(C=0.1, max_iter=10000))]
modelo=StackingClassifier(estimators=estimators,final_estimator=KNeighborsClassifier(n_neighbors=1, p=1))
f1 = cross_val_score(modelo, X_train,y_train,cv=5 ,scoring = 'f1_weighted').mean()
fit = modelo.fit(X_train,y_train)
pred = modelo.predict(X_test)
probs = modelo.predict_proba(X_test)
bien = []
for i in pred:
    if i == 0:
        bien.append([1,0,0,0])
    elif i == 25:
        bien.append([0,1,0,0])
    elif i == 50:
        bien.append([0,0,1,0])
    elif i == 75:
        bien.append([0,0,0,1])
bien = np.array(bien)
print('Los valores de las métricas para dicho modelo son los siguientes: ')
print(f'El brier Score da:',brier_multi(probs,bien))
print(f'El f1 predicho da:',f1_score(y_test,pred, average='weighted'))
print(f'F1 (Stacking): {f1:.5f}')

#### Probamos Bagging con los diferentes clasificadores

In [ ]:
modelo = BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=1, p=1),n_estimators=200)
f1 = cross_val_score(modelo, X_train,y_train,cv=5 ,scoring = 'f1_weighted').mean()
fit = modelo.fit(X_train,y_train)
pred = modelo.predict(X_test)
probs = modelo.predict_proba(X_test)

bien = []
for i in pred:
    if i == 0:
        bien.append([1,0,0,0])
    elif i == 25:
        bien.append([0,1,0,0])
    elif i == 50:
        bien.append([0,0,1,0])
    elif i == 75:
        bien.append([0,0,0,1])
bien = np.array(bien)
print('Los valores de las métricas para dicho modelo son los siguientes: ')
print(f'El brier Score da:',brier_multi(probs,bien))
print(f'El f1 predicho da:',f1_score(y_test,pred, average='weighted'))
print(f'F1 (Bagging): {f1:.5f}')

#### Probamos Boosting con los diferentes clasificadores

In [ ]:
modelo=AdaBoostClassifier(base_estimator=LogisticRegression(C=0.1, max_iter=100, penalty='l2',
                            random_state=10, solver='lbfgs'),n_estimators=200, algorithm='SAMME')
f1 = cross_val_score(modelo, X_train,y_train,cv=5 ,scoring = 'f1_weighted').mean()
fit = modelo.fit(X_train,y_train)
pred = modelo.predict(X_test)
probs = modelo.predict_proba(X_test)

bien = []
for i in pred:
    if i == 0:
        bien.append([1,0,0,0])
    elif i == 25:
        bien.append([0,1,0,0])
    elif i == 50:
        bien.append([0,0,1,0])
    elif i == 75:
        bien.append([0,0,0,1])
bien = np.array(bien)
print('Los valores de las métricas para dicho modelo son los siguientes: ')
print(f'El brier Score da:',brier_multi(probs,bien))
print(f'El f1 predicho da:',f1_score(y_test,pred, average='weighted'))
print(f'F1 (Boosting): {f1:.5f}')